In [14]:
from skimage import io
import numpy as np
import matplotlib.image as mimg
import cv2
from skimage.color import rgb2gray
import sys

In [15]:
def invertedBinaryImage(image):
    img = image.copy()
    shape = np.shape(img)
    if len(shape) > 2:
        if shape[2] > 3:
            img = rgb2gray(img[:,:,0:3])*255
        else:
            img = rgb2gray(img)*255
    b_img = np.zeros((shape[0], shape[1]))
    for v in range(0, shape[0]):
        for u in range(0, shape[1]):
            if img[v][u] != 255:
                b_img[v][u] = 1
    return b_img

In [16]:
def discreteContrast(filteredImage):
    shape = np.shape(filteredImage)
    img = filteredImage.copy()
    valueList = np.unique(img)
    valueDict = dict()
    for i in range(0, len(valueList)):
        valueDict.update({valueList[i] : i})
    for v in range(shape[0]):
        for u in range(shape[1]):
            img[v][u] = valueDict.get(filteredImage[v][u])
    return img

In [17]:
def invBiImg(img):
    ret_img = img.copy()
    shape = np.shape(img)
    b_img = np.ones((shape[0], shape[1]))
    for v in range(0, shape[0]):
        for u in range(0, shape[1]):
            if img[v][u] == 0:
                pass

In [18]:
def labelForeGround(image):
    img = rgb2gray(image)
    shape = np.shape(img)
    b_img = np.zeros((shape[0], shape[1]))
    for v in range(0, shape[0]):
        for u in range(0, shape[1]):
            if img[v][u] != 0:
                b_img[v][u] = 1
    return b_img

In [19]:
def invert(image):
    img = image.copy()
    img = (img * -1) + np.amax(image)
    return img

In [20]:
def labeledNeighbors(image, x, y, n=8):
    nbrs = list()
    if x - 1 >= 0 and image[y][x-1] > 1:
        nbrs.append((x-1, y))
    if n == 4:
        if y - 1 >= 0 and image[y-1][x] > 1:
            nbrs.append((x, y-1))
        return nbrs
    r = -1
    v = 2
    if x - 1 < 0:
        r = 0
    if x + 1 >= np.shape(image)[1]:
        v = 1
    if y - 1 >= 0:
        for i in range(r, v):
            if image[y-1][x+i] > 1:
                nbrs.append((x+i, y-1))
    return nbrs

In [21]:
def sequentialLabeling(image, n=8):
    # img = invertedBinaryImage(image)
    img = image.copy()
    m = 2
    c = list()
    for v in range(0, np.shape(img)[0]):
        for u in range(0, np.shape(img)[1]):
            if img[v][u] == 1:
                nbrs = labeledNeighbors(img, u, v, n)
                if len(nbrs) == 0:
                    img[v][u] = m
                    m = m + 1
                elif len(nbrs) == 1:
                    p = nbrs[0]
                    img[v][u] = img[p[1]][p[0]]
                elif len(nbrs) > 1:
                    p = nbrs.pop(0)
                    k = img[p[1]][p[0]]
                    img[v][u] = k
                    for p in nbrs:
                        ni = img[p[1]][p[0]]
                        if ni != k:
                            c.append({ni, k})
    
    r = list()
    for i in range(2, m):
        r.append({i})
    
    for collisions in c:
        a = collisions.pop()
        b = collisions.pop()
        for labelsets in r:
            if labelsets.issuperset({a}):
                ra = labelsets
            if labelsets.issuperset({b}):
                rb = labelsets
        if ra.isdisjoint(rb):
            ra.update(rb)
            r.remove(rb)
    
    for v in range(0, np.shape(img)[0]):
        for u in range(0, np.shape(img)[1]):
            if img[v][u] > 1:
                for lsets in r:
                    if lsets.issuperset({img[v][u]}):
                        img[v][u] = min(lsets)
    return invert(img)

In [22]:
def countAreaSize(sql_img):
    valueList, count, indices = np.unique(sql_img, return_counts=True, return_inverse=True)
    return valueList[0:len(valueList)], count[0:len(count)], indices[0:len(indices)]

In [23]:
def process(filename):
    img = io.imread(filename)
    bimg = labelForeGround(img)
    sql = discreteContrast(sequentialLabeling(bimg))
    labels, indices, counts = countAreaSize(sql)
    return removeClutter(counts)

In [24]:
def removeClutter(array, threshold=1000):
    ret = []
    for i in array:
        if i > threshold:
            ret.append(i)
    return ret

In [25]:
def regionSizeAndTone(labeledImage, colorImage):
    shape = np.shape(labeledImage)
    hsvImg = cv2.cvtColor(colorImage, cv2.COLOR_BGR2HSV)
    valueList, count = np.unique(labeledImage, return_counts=True)
    colorList = []
    for label in valueList:
        color = []
        for v in range(0, np.shape(img)[0]):
            for u in range(0, np.shape(img)[1]):
                if labeledImage[v][u] == label:
                    color.append(hsvImg[v][u][0])
        colorList.append(np.mean(color))
    return valueList, count, colorList

In [52]:
def regionSizeAndTone2():
    names = ['2Euro', '20Cent', '2Cent']
    size = 6
    sizeAndToneDict = dict()
    for n in names:
        mi  = sys.maxsize
        mx = 0
        color = []
        for i in range(0, size):
            labeledImage = sequentialLabeling(labelForeGround(io.imread(f'reference/highContrast/{n}{i}.png')))
            valueList, counts = np.unique(labeledImage, return_counts=True)
            count = np.max(removeClutter(counts[0:len(counts)-1]))
            colorImage = io.imread(f'reference/lowContrast/{n}{i}.png')
            hsvImg = cv2.cvtColor(colorImage, cv2.COLOR_BGR2HSV)
            shape = np.shape(labeledImage)
            for label in valueList:
                for v in range(0, np.shape(img)[0]):
                    for u in range(0, np.shape(img)[1]):
                        if labeledImage[v][u] == label:
                            color.append(hsvImg[v][u][0])
            if count <= mi:
                mi = count
            if count >= mx:
                mx = count
        sizeAndToneDict.update({n : (mi, mx, np.mean(color))})
    return sizeAndToneDict

In [ ]:
# nehme high und low contrast auf
# regionen filter auf high
## labelforeground
## sql
# für jede region farbe aus low ziehen und area size der region
# liste von tupeln(regiongröße, regionfarbe)


In [53]:
print(regionSizeAndTone2())

{'2Euro': (4449, 4873, 16.42557074652778), '20Cent': (3526, 3770, 42.90342447916667), '2Cent': (2445, 2598, 61.329631076388885)}


In [ ]:
def referenceValues():
    coinDict = dict()
    names = ['2Euro', '1Euro', '50Cent', '20Cent', '10Cent', '5Cent', '2Cent', '1Cent']
    for n in names:
        mi  = sys.maxsize
        mx = 0
        for i in range(0,5):
            filename = f'reference/highContrast/{n}{i}.png'
            image = Coinimage(io.imread(filename))
            image = image.highContrastToBinary()
            image = image.sequentialLabeling()
            labels, counts = image.countAreaSize()
            count = np.max(counts)
            if count <= mi:
                mi = count
            if count >= mx:
                mx = count
        coinDict.update({n : (mi, mx)})
    return coinDict

In [45]:
labeledImage = sequentialLabeling(labelForeGround(io.imread('test4hc.png')))
colorImage = io.imread('test4lc.png')
valueList, count, colorList = regionSizeAndTone(labeledImage, colorImage)
print(valueList)
print(count)
print(colorList)

[0. 2.]
[  4490 302710]
[107.20489977728285, 101.78076046381025]


In [41]:
img = io.imread('highcontrast/2Euro0.png')
img = labelForeGround(img)
img = sequentialLabeling(img)
img = discreteContrast(img)
labels, indices, counts = countAreaSize(img)
print(labels)
print(indices)
print(counts)

[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17.
 18. 19. 20. 21. 22. 23. 24. 25. 26. 27. 28. 29. 30. 31. 32. 33. 34. 35.
 36. 37. 38.]
[37 37 37 ... 38 38 38]
[  4953      2      1     15      6      1      3      1      6      2
      1      1      2      2      5      3      2      2      2      1
      1      1      6      3      1      4      1     14     12      1
     30      3      1      1     23      2      1      5 302079]


In [38]:
coinDict = dict()
mi  = 10000
mx = 0
names = ['2Euro', '1Euro', '20Cent', '10Cent', '5Cent']
for n in names:
    mi  = 10000
    mx = 0
    for i in range(0,5):
        filename = f'highcontrast/{n}{i}.png'
        # labels, counts, indices = process(filename)
        # pi = removeClutter(counts)
        pi = process(filename)
        for p in pi:
            if p <= mi:
                mi = p
            if p >= mx:
                mx = p
    coinDict.update({n : (mi, mx)})
for c in coinDict:
    print(c)
    print(coinDict.get(c))
    

2Euro
(4462, 5089)
1Euro
(3758, 4228)
20Cent
(3399, 3883)
10Cent
(2698, 3041)
5Cent
(3254, 3601)


In [19]:
names = ['2Euro', '1Euro', '20Cent', '10Cent', '5Cent']
for n in names:
    for i in range(0,5):
        filename = f'highcontrast/{n}{i}.png'
        print(filename)
        print(process(filename))
    

highcontrast/2Euro0.png
[4953]
highcontrast/2Euro1.png
[5089]
highcontrast/2Euro2.png
[4856]
highcontrast/2Euro3.png
[4462]
highcontrast/2Euro4.png
[4630]
highcontrast/1Euro0.png
[4182]
highcontrast/1Euro1.png
[4228]
highcontrast/1Euro2.png
[4113]
highcontrast/1Euro3.png
[3758]
highcontrast/1Euro4.png
[3902]
highcontrast/20Cent0.png
[3883]
highcontrast/20Cent1.png
[3873]
highcontrast/20Cent2.png
[3740]
highcontrast/20Cent3.png
[3399]
highcontrast/20Cent4.png
[3505]
highcontrast/10Cent0.png
[3037]
highcontrast/10Cent1.png
[3041]
highcontrast/10Cent2.png
[2976]
highcontrast/10Cent3.png
[2698]
highcontrast/10Cent4.png
[2795]
highcontrast/5Cent0.png
[3578]
highcontrast/5Cent1.png
[3601]
highcontrast/5Cent2.png
[3485]
highcontrast/5Cent3.png
[3254]
highcontrast/5Cent4.png
[3379]


In [42]:
import src.hufftransform
src.hufftransform.HuffTransformCounter.run(image=io.imread(f'reference/highContrast/2Euro1.png'))

TypeError: 'NotImplementedType' object is not callable

In [43]:
import src.lib as l
coinimg = l.Coinimage(io.imread(f'reference/highContrast/2Euro1.png'))
img = coinimg.image
b_img = coinimg.highContrastToBinary()
b_img2 = b_img.image
b_img3 = labelForeGround(img)
b_img4 = l.highContrastToBinary(img)
sql4 = l.sequentialLabeling(b_img4)
sql3 = sequentialLabeling(b_img3)
sql = b_img.sequentialLabeling()
img2 = sql.image
x,y =np.unique(img2, return_counts=True)
print(np.max(y[0:len(y)]))
print(np.unique(b_img2))
print(np.unique(img2))
print(np.unique(sql3))
print(np.unique(b_img4))
print(np.unique(sql4))

AttributeError: module 'src.lib' has no attribute 'highContrastToBinary'